# Welcome to the introduction to machine learning workshop!!!

This is a Python notebook for reference. Feel free to set this up on Google Colab (https://colab.research.google.com/) or your local machine with your favorite IDEs, and follow along with the presentation! 

Once you have the dataset, run the cell below to read in the data and test out all the model demos. The markdown cells will explain what the codes are doing, but please ask questions if anything is unclear, or you run into issues. There are no stupid questions! 

## Categories of ML Tasks

Machine Learning tasks can be generalized into three categories. 

| Supervised | Unsupervised | Reinforcement |
| ---------- | ------------ | ------------- |
| A model is trained given the input features, X, and their corresponding targets, Y | A model is trained given only the input features, X, without their corresponding targets | Train a model or an action policy of an agent based on feedback from the environment | 
| regressions tasks, labeled classification tasks | clustering, dimensionality reduction |  markov decision process, Q table learning |


In this workshop, we will look at both supervised and unsupervised tasks, including: 
* Stock Price Prediction 
* Image Classification
* Anomaly Detection

There are no reinforcement learning tasks in this workshop, but it will be covered in future workshops, stay tuned! 😁

<b>At the end of this workshop, we will host a mini-competition using another image classification dataset. There will be prizes for top contenders!</b>

## Steps to solve a ML task

<h4>We follow the guideline of the "Machine Learning Life Cycle" when building our model.</h4>

<!-- ![ML-Lifecycle](ML-lifecycle.drawio.png){ width="800" height="600" style="display: block; margin: 0 auto" } -->
<p align="center">
    <img src="ML-lifecycle.drawio.png">
<p>

In reality, designing and implementing data pipeline and making model production-ready is a lot more time consuming than model building; we will cover these aspect of ML in other workshops in the future. 

However, in this workshop, we already collected the data for you, and there will not be any model deployment required. We will focus on the `training` and `evaluation` of a model. 


## Stock Price Prediction

Algorithmic trading is an extremely popular field. Many of you probably heard of terms such as 'quantitative trading' or 'quantitative research' while looking for your next internship/job. `Quantitative Researchers/Traders` (quants for short) differ from traditional traders because they relies on algorithms, which is constructed based off of models, to achieve trading objectives, such as high frequency trading and arbitrage trading. 
<br>

To create effective trading strategies, a trader must leverage as much predictive power as possible. For the first example, we are attempting to model a stock's trend with ML, and see if this model is effective in predicting the future price. 
<br>

Think about what type of ML task would this be? 
* Supervised or Unsupervised? 
* Regression or Classification? 

## Image Classification 

Image classification is usually the first thing people think of when taking about computer vision, and we can see image classification model at work on a daily basis. For the second example, we will construct an effective image classification model to identify hand written digits from their picture. 

## Anomaly Detection

Anomaly detection is a category of tasks. 